In [1]:
import checklist
import math
import nltk
import numpy as np
import random
import spacy

from checklist.perturb import Perturb
from datasets import load_dataset, dataset_dict
from progress.bar import ShadyBar
from typing import Tuple

In [2]:
nlp = spacy.load("en_core_web_sm")
data_set = load_dataset('cnn_dailymail', '3.0.0')

Reusing dataset cnn_dailymail (/home/philko/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)
/home/philko/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
type(data_set)

datasets.dataset_dict.DatasetDict

In [4]:
len(data_set['train']) + len(data_set['validation']) + len(data_set['test'])

311971

In [5]:


def concat_data_set(data_set : dataset_dict.DatasetDict, cutoff_fun : callable) -> list:

    data : list = list()

    bar = ShadyBar('Creating dataset', max=len(data_set['train']) + len(data_set['validation']) + len(data_set['test']))

    for e in data_set['train']:
        data.append(
            cutoff_fun(e)
        )
        bar.next()

    for e in data_set['validation']:
        data.append(
            cutoff_fun(e)
        )
        bar.next()

    for e in data_set['test']:
        data.append(
            cutoff_fun(e)
        )
        bar.next()
    bar.finish()

    return data

cnn_cutoff = lambda elem : {'article': elem['article'], 'id': elem['id']}
# concat_data_set(data_set, cnn_cutoff)

In [6]:
    def negate_data(sentences : list) -> Tuple[list, list]:
        """ Perturbation function

        Complexity is O(n^2).

        Parameters
        ----------
        text : str
            text, passed as a string.
        
        Returns
        -------
        list
            list of sentences

        Raises
        ------
        Exception
            if percentage is not in the interval [0,1]
        """
        return_list : list = list()
        not_negated : list = list()

        for index, sentence in enumerate(list(nlp.pipe(sentences))):
            sent_tmp : str = sentence.text
            try:
                ret = Perturb.perturb([sentence], Perturb.add_negation, keep_original=False)
                if len(ret.data) > 0:
                    sent_tmp = ret.data[0][0]
                else:
                    not_negated.append(index)
                    print("Couldn't find proper negation.")

            except TypeError:
                not_negated.append(index)
                print("Couldn't find proper negation.")

            return_list.append(sent_tmp)


        return return_list, not_negated 

In [7]:
def tokenize_sentences(text : str) -> list:
        return nltk.sent_tokenize(text)

In [8]:
data = data_set['train'][0]['article']

In [9]:
sentences = tokenize_sentences(data)

In [11]:
negate_data(sentences=sentences)
data

Couldn't find proper negation.
Couldn't find proper negation.
Couldn't find proper negation.
Couldn't find proper negation.
Couldn't find proper negation.
Couldn't find proper negation.
Couldn't find proper negation.
Couldn't find proper negation.
Couldn't find proper negation.


'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he want

In [12]:

    def createRepetitions(\
            sentences : list,\
            doc : spacy.tokens.doc.Doc,\
            sent_ind : int,\
            phraseLength : int,\
            nTimes : int) -> bool:
        """ Creating Repetitions in one sentence

        Function to create repetitions in one sentence. To avoid 
        the repitition of punctations, only phrase without punctuations 
        will be choosen. Alteration is done inplace.
        Complexity is O(n).

        Parameter
        ---------
        sentences : list
            list of sentences in which one sentence will be perturbated
        doc : pacy.tokens.doc.Doc
            parsed tokens as a spaCy doc
        sent_ind : int
            index of sentence to be perturbated
        phraseLength : int
            length of a phrase to be repeated
        nTimes : int
            number of times the phrase will be repeated
        """
        # subtract 1 because of indexing
        for i in reversed(range(phraseLength - 1, len(doc))):
            token_slice = doc[(i - phraseLength):i]
            if not True in [token.pos_ == 'PUNCT' for token in token_slice]:

                index = doc[i].idx

                rep = " ".join([token.text for token in token_slice])
                further_tokens = " ".join([token.text for token in doc[i:len(doc)]])
                sentences[sent_ind ] = sentences[sent_ind ][0:index] + " " + rep + further_tokens

                #print(f"Repetition for a phrase with %i words %i times added. Sentence No.: %i" % (phraseLength, nTimes, sent_ind))
                return True
        return False



    def repeat_words(sentences : list, nTimes : int = 3, phraseLength : int = 4):
        """ Repeat wordssfunction

        Function to repeats some words in every sentence of the text. With the passed probability, a sentence in the text is perturbated.
        Complexity is O(n^2) (function call).

        Parameter
        ---------
        sentences : list
            list of already tokenized sentence tokens
        nTimes : int
            number of repetitions of each phrase
        phraseLength : int
            length of phrase to be repeated
            
        Returns
        -------
        list, list
            list of sentences, list of sentences where a phrase is repeated

        """

        indices : list = []
        for i in range(len(sentences)):
            
            tokens = nlp(sentences[i])

            if len(tokens) <= phraseLength:
                continue

            if createRepetitions(sentences=sentences, doc=tokens, sent_ind =i, phraseLength=phraseLength, nTimes=nTimes):
                indices.append(i)

        return sentences, indices

In [13]:
repeat_words(sentences=sentences)

(["It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria military force in Syria.",
  'Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons use of chemical weapons.',
  'The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction weapons of mass destruction. "',
  "It's a step that is set to turn an international crisis into a fierce domestic political battle fierce domestic political battle.",
  'There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria inspectors find in Syria?',
  'What happens if Congress votes no if Congress votes no?',
  'And how will the Syrian gover

In [14]:
def swap_pair(sentence : str, doc : spacy.tokens.doc.Doc) -> str:
    """ Swap pair function

    Function to swap one random pair of words. Using the random sample function,
    two elements are choosen and swaped later on.

    Parameter
    ---------
    sentence : str
        sentence to be deteriorated
    doc : spacy.tokens.doc.Doc
        spacy document, to extract the indices of the tokens from

    Returns
    -------
    str
        deteriorated sentence
    """
    
    candidates : list = []
    candidates_text : list = []

    for i in range(len(doc)):

        lower_text = doc[i].text.lower()

        if doc[i].pos_ != "PUNCT" and not lower_text in candidates_text:
            candidates.append(i)
            candidates_text.append(lower_text)
        else:
            continue

    pair : list = random.sample(candidates, 2)
    first, second = (pair[0], pair[1]) if pair[0] < pair[1] else (pair[1], pair[0])
 
    first_bounds, second_bounds = \
        (doc[first].idx, doc[first].idx + len(doc[first].text)), \
        (doc[second].idx, doc[second].idx + len(doc[second].text))

    first_token, second_token = \
        sentence[first_bounds[0]:first_bounds[1]], \
        sentence[second_bounds[0]:second_bounds[1]]
    
    return sentence[0:(first_bounds[0])] + second_token + " " +\
        sentence[(first_bounds[1] + 1):(second_bounds[0])] + first_token + \
        sentence[(second_bounds[1])::]

    

def word_swap(sentences : list):
    """ Word swap function

    Function to swap words in every sentence as far as it is possible. 

    Parameter
    ---------
    sentences : list
        list of already tokenized sentence tokens

    Returns
    -------
    list, list
        list of sentences, list of indices of sentences with a swapped word
    """
    ret_list : list = []
    indices : list = []

    for i, sentence in enumerate(sentences):

        times : int = random.randrange(1, 5)
        new_sentence = sentence

        for _ in range(times):
            new_sentence = swap_pair(sentence=new_sentence, doc=nlp(new_sentence))

        ret_list.append(new_sentence)
        indices.append(i)
    return ret_list, indices

In [15]:
word_swap(sentences)

(["It's official: U.S. President Barack Obama wants lawmakers use weigh in military whether to to on force in Syria military force in Syria.",
  'Obama sent a letter to announcing heads of the House and Senate on Saturday night, hours after the that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons use of chemical weapons.',
  'The proposed other from or asks Congress to approve the use of military deter "to force, disrupt, prevent and degrade the potential for future uses of chemical weapons Obama legislation weapons of mass destruction weapons of mass destruction. "',
  "It's a step that crisis set to turn an international is fierce a into domestic political battle fierce domestic political battle.",
  'There debate key questions looming over the are: What inspectors U.N. weapons did find in Syria inspectors find in Syria?',
  'What if happens Congress votes no if Congress votes no?',
  'how And the will Syrian gover

In [16]:
def drop_single(sentence : str, doc : spacy.tokens.doc.Doc) -> list:
    """ Drop single word function

    Function to drop a single word from a sentence. 

    Parameter
    ---------
    sentence : str
        sentence to be deteriorated
    doc : spacy.tokens.doc.Doc
        spacy document, to extract the indices of the token from

    Returns
    -------
    str
        deteriorated sentence
    """

    candidates : list = []

    for i in range(len(doc)):

        if doc[i].pos_ != "PUNCT":
            candidates.append(i)
        else:
            continue
    
    to_drop : int = random.randrange(0, len(doc))

    bounds = doc[to_drop].idx, doc[to_drop].idx + len(doc[to_drop].text)

    return sentence[0:bounds[0]] + sentence[(bounds[1] + 1)::]
    


def word_drop(text : str):
    """ Word drop function

    Function to drop random words from a sentence.

    Parameter
    ---------
    sentences : list
        list of already tokenized sentence tokens

    Returns
    -------
    list
        list of sentences
    """


    ret_list : list = []
    indices : list = []

    for i, sentence in enumerate(sentences):

            times : int = random.randrange(1, 5)
            new_sentence = sentence

            for _ in range(times):
                new_sentence = drop_single(sentence=new_sentence, doc=nlp(new_sentence))

            ret_list.append(new_sentence)
            indices.append(i)
    return ret_list, indices

In [17]:
word_drop(sentences)

(['s official: U.S. Barack Obama wants to weigh in on whether to use military force in Syria force in Syria.',
  'Obama sent a letter to the heads of the House and Senate on Saturday ours after announcing that he believes military action against Syrian targets is the right step take over the alleged use of chemical weapons use of chemical weapons.',
  'The proposed legislation from Obama asks Congress to approve the use of military force "to deter,  prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction weapons of mass destruction. "',
  'Ita step that is set turn an international crisis into a fierce domestic political battle fierce domestic political battle.',
  'There are key questions over debate: What did U.N. weapons inspectors find in inspectors find Syria?',
  'happens Congress votes Congress votes no?',
  'how will Syrian react the Syrian government react?',
  'In a televised address from the White House Rose Garden earlier S

In [18]:
    def drop_single_pos(sentence : str, doc : spacy.tokens.doc.Doc, pos : str) -> list:
        """ Drop single word function

        Function to drop words based on their POS tag. 

        Parameter
        ---------
        sentence : str
            sentence to be deteriorated
        doc : spacy.tokens.doc.Doc
            spacy document, to extract the indices of the token from
        pos : str
            pos tag whose words will be dropped

        Returns
        -------
        str
            deteriorated sentence
        """

        candidates : list = []

        for i in range(len(doc)):

            if doc[i].pos_ == pos:
                candidates.append(i)
            else:
                continue
        
        if len(candidates) == 0:
            return sentence, False
        
        diff : int = 0
        for i in candidates:
            bounds = doc[i].idx - diff, doc[i].idx + len(doc[i].text) - diff
            sentence = sentence[0:bounds[0]] + sentence[(bounds[1] + 1)::]
            diff += len(doc[i].text) + 1
        

        return sentence, True
        


    def pos_drop(text : str,  probability : float, pos : str):
        """ POS drop function

        Function to drop random words with a specific POS-Tag from a sentence.

        Parameter
        ---------
        sentences : list
            list of already tokenized sentence tokens
        probability : float
            float in [0,1]
        pos : str
            pos tag which occurences will be dropped

        Raises
        ------
        Exception
            if probability is not in [0,1]
        
        Returns
        -------
        list
            list of sentences
        """

        if probability < 0 or probability > 1:
            raise Exception("Probability must be a number in [0,1].")

        sentences = nltk.sent_tokenize(text)

        ret_list : list = []
        indices : list = []

        # TODO pipe sentences
        for i, sentence in enumerate(sentences):
                if bool(np.random.binomial(size=1, n=1, p= probability)):
                    new_sentence, success = drop_single_pos(sentence=sentence, doc=nlp(sentence), pos=pos)

                    if success:
                        indices.append(i)
                    ret_list.append(new_sentence)
                else:
                    ret_list.append(sentence)
    
        return  ret_list, indices

In [24]:
from spacy import displacy
dropped, number_dropped = pos_drop(data, 0.5, "ADJ")
print(dropped, number_dropped)

test = nlp(sentences[2])
#displacy.serve(test, style="dep")

["It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria.", 'Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons.', 'The proposed legislation from Obama asks Congress to approve the use of force "to deter, disrupt, prevent and degrade the potential for uses of weapons or weapons of destruction."', "It's a step that is set to turn an crisis into a battle.", 'There are questions looming over the debate: What did U.N. weapons inspectors find in Syria?', 'What happens if Congress votes no?', 'And how will the government react?', 'In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he wants to.', '"While I believe I have the authority to carry out this ac

In [20]:
len(data)

9396

In [23]:
len(sentences)

76

In [25]:
len(number_dropped)

31

In [26]:
data_set['train'][[1,2,3]]

{'article': ['(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men\'s 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I\'m proud of myself and I\'ll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory